## Load Libraries

In [6]:
import cv2
import numpy
import imutils
import os
import pickle

import matplotlib.pyplot as plt
from paddleocr import PaddleOCR
ocr_model = PaddleOCR(lang='en', show_log=False, use_angle_cls=True, use_gpu=True, enable_mkldnn=True)


## Generate the Hexadecimal for Positive Samples

In [8]:
# Each product should have a product number which represent it's ID.
# Let's generate the product number for the label

hex_start = int("73de94dc0fd", 16) # Mac address first
hex_last = int("73DE94DC160", 16) # Mac address last

hexa_list = list()
for i in range(hex_start,hex_last+1):
    hex_cal = hex(i).upper()
    hex_cal = hex_cal[2:].upper()
    #print(f"deximal {i} with hex {hex_cal}")
    hexa_list.append(hex_cal)
    
print(f" Generated {len(hexa_list)} Product Numbers")
print("--DONE--!")

 Generated 100 Product Numbers
--DONE--!


## Generate the Hexadecimal for Negative Samples

In [11]:

hex_start_n = int("73DE94DC166", 16) # Mac address first
hex_last_n = int("73DE94DC176", 16) # Mac address last

hexa_list_n = list()
for j in range(hex_start_n,hex_last_n+1):
    hex_cal_n = hex(j).upper()
    hex_cal_n = hex_cal_n[2:].upper()
    #print(f"deximal {i} with hex {hex_cal}")
    hexa_list_n.append(hex_cal_n)
    
print(f" Generated {len(hexa_list_n)} Product Numbers")
print("--DONE--!")

 Generated 17 Product Numbers
--DONE--!


## Save Positive Labels Product Numbers

In [5]:
## Save our Data in a pickle file
import pickle

data = {"prod_numb":hexa_list}

with open("label_db.pickle", "wb") as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Create our Positive Labels

In [10]:
# Create a label folder if does not exist
label_folder = "./positive_samples"
if not os.path.isdir(label_folder):
    os.mkdir(label_folder)
    
for i in hexa_list:
    image = cv2.imread("box_label_image.PNG")
    product_number = f"Prod Number: {i}"
    image = cv2.putText(image, product_number,  (775,1012), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,0,0), 2, cv2.LINE_AA)
    label_name = f"{label_folder}/label_{i}.png"
    cv2.imwrite(label_name, image)

## Create our Negative Labels

In [12]:
# Create a label folder if does not exist
negative_samples_folder = "./negative_samples"
if not os.path.isdir(negative_samples_folder):
    os.mkdir(negative_samples_folder)
    
for i in hexa_list_n:
    image = cv2.imread("Capture.PNG")
    product_number = f"Prod Number: {i}"
    image = cv2.putText(image, product_number, (775,1012), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,0,0), 2, cv2.LINE_AA)
    label_name = f"{negative_samples_folder}/label_{i}.png"
    cv2.imwrite(label_name, image)

## Inference

In [5]:
# Create a function to extract our product number
def extract_prod_num(image):

    result = ocr_model.ocr(image)
    ocr_result = [line[1][0] for line in result[0]]
    prod_num = [i for i in ocr_result if i.startswith("Prod ")]
    prod_num = prod_num[0].split(": ")[1]
    return prod_num

# define a function to verify the correctness of the extracted product number
def verify_correctness(prod_num):
    DB_file= open('label_db.pickle', 'rb')
    DB_data = pickle.load(DB_file)
    # close the file
    DB_file.close()
    
    if prod_num in DB_data['prod_numb']:
        return "The Product Number Exist in our Database"
    else:
        return "The Product Number does not Exist in our Database"
    
extracted_prod_num = extract_prod_num("./labels_folder/label_73DE94DC160.png")
print(f"The Product Number is {extracted_prod_num}")
        
final_result = verify_correctness(extracted_prod_num)
print(final_result)


The Product Number is 73DE94DC160
The Product Number Exist in our Database


'The Product Number Exist in our Database'

# UTILITIES

## Get Coordinates

In [ ]:
# import the required library
import cv2

# define a function to display the coordinates of

# of the points clicked on the image
def click_event(event, x, y, flags, params):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(f'({x},{y})')
      
        # put coordinates as text on the image
        cv2.putText(img, f'({x},{y})',(x,y),
        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
      
      # draw point on the image
        cv2.circle(img, (x,y), 3, (0,255,255), -1)
 
# read the input image
#img = cv2.imread('image_size.png')
img = cv2.imread("C:/Users/savadogo_abdoul/Desktop/Learn/box_data/Capture.PNG")
#img = cv2.resize(image, (640, 480))

# create a window
cv2.namedWindow('Point Coordinates')

# bind the callback function to window
cv2.setMouseCallback('Point Coordinates', click_event)

# display the image
while True:
    cv2.imshow('Point Coordinates',img)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
cv2.destroyAllWindows()

(775,959)
(1304,957)
(1307,1017)
(771,1012)
